# Costas Loop
#### This version of the Costas Loop was designed by mirroring the block diagram (Borre, pg 94) rather than using feedback loop design techniques. Each cell represents an block in the diagram.

## Initialization of variables

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import math
import string
#Store the shift registers as a deque, so that deque.rotate{} can be used.
from collections import deque
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from GoldCode import GoldCode

# Initializations

ms = 10
SampleLength = ms*10**(-3)
phi = 0.0              # Original signal phase
phi_hat = 0.3      # phi_hat is the output phase - I channel
phi_hat_Q = phi_hat + np.pi/2    # phi_hat_Q is the output phase - Q channel
freq_offset = 0.3
delta_phi = 0.0    # delta_phi is the phase difference between the incomming signal and the VCO
wn = 0.01          # wn is omega_n, pll bandwidth
zeta = 0.707       # Damping factor of the loop
K = 1000           # K = loop gain, which I assume can be chosen, but this is a sample value
N = 1023           # Number of samples

# Set tap for a particular satellite
tap = (1, 5)

# These are just initializing some vectors for plotting
ref = np.linspace(0, N, N, dtype=int)

## "Incoming signal" block

In [5]:
#def tstSignalGen(ref, phi, freq_offset):
#    signal = []       # Save the signal_in for plotting
#    for i in ref:
#
#        # Complex input signal
#        signal_in = np.cos(phi) + 1j*np.sin(phi)
#        phi += freq_offset

#        signal.append(signal_in) # Saved for plotting
#    return signal

In [22]:
def tstSignalGen(self):
    f = 10 
    Fs = 1023*4*10**3
    sample = Fs
    x = np.arange(sample)
    self.t = np.array(x/Fs)
    tstSignal = np.sin(2 * np.pi * f * t)
    plt.plot(t, tstSignal)
    plt.show()
    #print(len(x))
    #print(len(tstSignal))
    #print(len(t))
    return tstSignal

## "PRN code" block
#### Uses the GoldCode class

In [23]:
Satellite = 1

# Create list of C/A code Taps, for simpler sat selection",
#sat = [(2,6),(3,7),(4,8),(5,9),(1,9),(2,6),(1,8),(2,9),(3,10),(2,3),(3,4),(5,6),(6,7),(7,8),(8,9),(9,10),(1,4),(2,5),(3,6),(4,7),(5,8),(6,9),(1,3),(4,6),(5,7),(6,8),(7,9),(8,10),(1,6),(2,7),(3,8),(4,9),(5,10),(4,10),(1,7),(2,8),(4,10)]
sat = [(1,5),(2,6),(3,7),(4,8),(0,8),(1,5),(0,7),(1,8),(2,9),(1,2),(2,3),(4,5),(5,6),(6,7),(7,8),(8,9),(0,3),(1,4),(2,5),(3,6),(4,7),(5,8),(0,2),(3,5),(4,6),(5,7),(6,8),(7,9),(0,5),(1,6),(2,7),(3,8),(4,9),(3,9),(0,6),(1,7),(3,9)]

# Create Code Generator object for chosen Satellite
CodeGen = GoldCode(sat[Satellite - 1]) # Index starts at zero

# Generate CA Code
CACode = CodeGen.getCode(1023)

# Repeat each chip 4 times (See markdown in above cell), to match our ADC sample frequency",
CACodeSampled = np.repeat(CACode,4)

# Repeat entire array for each ms of data sampled
CACodeSampled = np.tile(CACodeSampled,int(SampleLength*1000))

print(CACodeSampled)
print(len(CACodeSampled))

[ 1  1  1 ..., -1 -1 -1]
40920


## "NCO carrier generator" and "90 degree" blocks
#### These two blocks can be combined because they can really be thought of as one block, and they are simple, especially the "90 degree" block.

In [24]:
for sample in signal:

    # Complex oscillator signal - I channel
    vcoI = np.cos(phi_hat) + 1j*np.sin(phi_hat)
    # Complex oscillator signal - Q channel
    vcoQ = np.cos(phi_hat_Q) + 1j*np.sin(phi_hat_Q)

    delta_phi = np.angle(sample * np.conj(vcoI))
    output.append(vcoI)

    # vx varibles are a buffer to store output changes on transfer function variables ax and bx
    phi_hat =  v0 * b0 + v1 * b1 + v2* b2
    phi_hat_Q = increment
return output


NameError: name 'signal' is not defined

## "Carrier loop discriminator" block
#### There are three possible discriminators as described in Borre, pg 94-95, two of which are included but commented out. 

In [25]:
# psi = np.arctan(Q/I) # This is the most precise discriminator, but it also takes the most time.
# psi = I*Q # This discriminator is the fastest, but least precise
# psi = math.sign(I) * Q

## Multiply "Incoming signal" and PRN code" blocks

In [27]:
satCode = GoldCode(tap)
code = satCode.getCode(1023)
tstSignal = np.array(tstSignalGen(self))
# Repeat each chip 4 times (See markdown in above cell), to match our ADC sample frequency",
CACodeSampled = np.repeat(code,4)
print("Satellite chosen: %d, with tap: %s" %(Satellite,str(sat[Satellite - 1])))

# Repeat entire array for each ms of data sampled
CACodeSampled = np.tile(CACodeSampled,int(SampleLength*1000))

INxPRN = tstSignal[0:40920] * CACodeSampled
#print(satCode)
#print()
#print(code)
#print()
#print(tstSig)
#print()
#print(INxPRN)

print(len(CACodeSampled))
print(len(tstSignal))

#plt.plot(ref[1:100], INxPRN[1:100])
#plt.show()
#plt.plot(ref[1:100], code[1:100])
#plt.show()
#plt.plot(ref[1:100], tstSig[1:100])
#plt.show()

NameError: name 'self' is not defined

## Mutiply ["NCO carrier generator" x ("Incoming signal" x "PRN code")]

In [ ]:
# INxPRN * NCOI 
# as well as
# INxPRN * NCOQ

## "Lowpass filter" blocks
#### Considering the filter only needs to handle 2f, a simple first-order filter seems to be sufficient

In [ ]:
# scipy.signal.butter()

## Carrier loop filter" block